# GETTING AND LOADING A CORPUS

https://nlpforhackers.io/corpora/
http://lucumr.pocoo.org/2015/11/18/pythons-hidden-re-gems/

The goal of this step is to develop an initial list of each character and their spoken lines. Dictionaries are Hash value arbitrary, so may not be ordered the same.

In [1]:
#Select and Read a file into "f" using a list and stripping out all Project Gutenberg headers and footers
from pathlib import Path
import re
idx = 1
data_folder = Path("data/shakespeare/")
file_to_open = data_folder / "1526.txt"
f = open(file_to_open, 'r')
first_document = list(f)

for first_header_index in range( len(first_document) ):
    if ( first_document[first_header_index].startswith('a team of about twenty Project Gutenberg volunteers.\n') ) :
           break
            
second_document = list(first_document[first_header_index + 1 :])

for second_header_index in range( len(second_document) ):
    if ( second_document[second_header_index].startswith('a team of about twenty Project Gutenberg volunteers.\n') ) :
           break            
for footer_index in range( len(first_document) ):
    if ( first_document[footer_index].startswith('End of Project Gutenberg Etext of') ) :
           break
            
            
script = list()
script = list(first_document[first_header_index + second_header_index + 2 : footer_index-1])

#print(script)

In [2]:
#Compile a list of speakers
r = re.compile("[A-Z0-9][A-Z0-9]+")
speakers = []
for line in script:
    mtch = r.match(line)
    if mtch:
        speakers.append(mtch.group())
#print(speakers)

#Omit speakers from the list of text
s = re.compile(r"\b[A-Z{3}\.]+\b")
spoken = list(filter(lambda i: not s.search(i), script))

#print(spoken)

In [8]:
#Concatenate lines into list entries for future sentence splitting

newLines = []
singleLine = ''
singleLines = []

#Remove all line returns(ok)
for j in range(0, len(spoken)):
    spoken[j] = spoken[j].replace('\n', '')
    
#Split 5 lines at a time into new list
for k in range( 0, len(spoken), 2):
    newLines = []
    for line in range( 0, 5 ):
        try:
            newLines.append(' '+spoken[line+k])
        except:
            print("Index Error at", k, line)
            break
    #Join 5-line groups into one line and append to a list
    singleLine = ''.join(newLines)
    singleLines.append(singleLine)

#print(singleLines)

Index Error at 2890 4
Index Error at 2892 2
['     ', '     by William Shakespeare', '   by William Shakespeare  ', ' by William Shakespeare    ', '     ', '     ', '    Lords, Priests, Sailors, Officers, Musicians, and other Attendants.', '  Lords, Priests, Sailors, Officers, Musicians, and other Attendants.  ', ' Attendants.    ', '     ', '     If music be the food of love, play on,', '   If music be the food of love, play on, Give me excess of it; that, surfeiting, The appetite may sicken and so die.--', ' If music be the food of love, play on, Give me excess of it; that, surfeiting, The appetite may sicken and so die.-- That strain again;--it had a dying fall; That breathes upon a bank of violets,', " The appetite may sicken and so die.-- That strain again;--it had a dying fall; That breathes upon a bank of violets, Stealing and giving odour.--Enough; no more; 'Tis not so sweet now as it was before.", " That breathes upon a bank of violets, Stealing and giving odour.--Enough; no m

In [40]:
#Create list of sentences
sentences = []
for m in range(0, len(singleLines)):
    mtch = re.findall("[A-Z][^\.!?]*[\.!?]", singleLines[m], re.M|re.I)
    if mtch:
        sentences.append(mtch)
print(sentences)

[['Lords, Priests, Sailors, Officers, Musicians, and other Attendants.'], ['Lords, Priests, Sailors, Officers, Musicians, and other Attendants.'], ['Attendants.'], ['If music be the food of love, play on, Give me excess of it; that, surfeiting, The appetite may sicken and so die.'], ['If music be the food of love, play on, Give me excess of it; that, surfeiting, The appetite may sicken and so die.'], ['The appetite may sicken and so die.', 'That strain again;--it had a dying fall; That breathes upon a bank of violets, Stealing and giving odour.', "Enough; no more; 'Tis not so sweet now as it was before."], ['That breathes upon a bank of violets, Stealing and giving odour.', "Enough; no more; 'Tis not so sweet now as it was before."], ['Tis not so sweet now as it was before.'], ['Receiveth as the sea, nought enters there, Of what validity and pitch soever, But falls into abatement and low price Even in a minute!', 'so full of shapes is fancy, That it alone is high-fantastical.'], ['But 

In [ ]:
#Create list of phrases
phrases = []

In [ ]:
import re
# Import defaultdict
from collections import defaultdict

freqdict = defaultdict(int)
for key, value in dictionary.items():
    utterance = dictionary[spoken(key)]
    cleantext = re.sub(r'\n', '', utterance) #.strip().split() # Remove HTML tags and split the review by word (space separated)
    for word in cleantext:
        # Convert to all lowercase
        word = word.lower()
    
        # Complete the following line to increase the count by one:
        freqdict[word] += 1
    
print(freqdict)

# TF-IDF

https://cdn-images-1.medium.com/max/800/1*nSqHXwOIJ2fa_EFLTh5KYw.png

In [ ]:
# Computes ratio of word's appearances to total words
def computeTF(wordDict, bow): 
    bowCount = len(bow) #BOW = Bag of Words
    tfDict = {}
    for word, count in wordDict.items():
        tfDict[word] = count/float(bowCount)
    return tfDict

In [ ]:
# Calculates the weight of rare words across all docs
def computeIDF(docList):
    import math
    idfDict = {}
    N = len(docList)
    
    idfDict = dict.fromkeys(docList[0].keys(), 0)
    for doc in docList:
        for word, val in doc.items():
            if val > 0:
                idfDict[word] += 1
                
    for word, val in idfDict.items():
        idfDict[word] = math.log10(N / float(val))
    
    return idfDict

In [ ]:
def computeTFIDF(tfBow, idfs):
    tfidf = {}
    for word, val in tfBow.items():
        tfidf[word] = val*idfs[word]
    return tfidf